# Calculating sums and averages
Workflow that describes how the statistics over a set of intervals / zones can be calculated in Python

Necessary imports

In [1]:
import os
import utils.io as uio
import core.well as cw
from utils.calc_stats import calc_stats2 as calc_stats
from core.well import Project

## Create a wells project

In [2]:
wp = Project(name='MyProject', tops_file='test_data/RokDocTops.xlsx', tops_type='rokdoc')

*Project* also take keywords:
- *working_dir* - the folder where the project is situated
- *project_table* - full or relative path name of a specific formatted .xlsx table that contain important project information
- *log_to_stdout* - if True, logging information is sent to standard output, else to a text file

By default, none of these need to set, and it uses the install directory, toghether with example data and project table there

## Load project table
The project table is a .xlsx file that contain, among other things, which wells and well logs to load, and where to find them.
If some wells have different names for the same well log, you need to *harmonize* the log names so that they are understood correctly in the project.

For example, *Well_F* in case, uses *VSH* as name for shale volume, while the others use *VCL*, to rename from *VSH* to *VCL*, use the following 

In [3]:
#well_table = uio.project_wells(wp.project_table, wp.working_dir)
wells = wp.load_all_wells({'VCL': ['VSH']})

0 WELL_A test_data/Well A.las
1 WELL_B test_data/Well B.las
2 WELL_C test_data/Well C.las


In [4]:
#for well in wells.values():
#    print(well.log_names())
#    well.depth_plot('Volume')

## Load tops
Load file with well tops. They can be in format exported from Petrel (use uio.read_petrel_tops()), npd.no (use uio.read_npd_tops()), or RokDoc (use uio.read_rokdoc_tops())


In [5]:
tops = uio.read_tops(wp.tops_file, frmt=wp.tops_type)
#print('The tops file contains following wells and tops:')
for well in list(tops.keys()):
    print(' Well: {}'.format(well))
#    for ttop in list(tops[well].keys()):
#        print('  - {}'.format(ttop))

 Well: WELL_A
 Well: WELL_B
 Well: WELL_F
 Well: WELL_C


   # Calculate RokDoc compatible Sums & Averages
  This functionality is useful when you want to analyze the statistics of several formations / intervals across multiple wells (.las files).
  The results is saved as an .xlsx spreadsheet, with statistics of the rock properties. To be read by RokDoc, it has to be converted to .xls

Tell python where to save the results, where to find the wells, and the tops file

If the .xlsx file exists, it will be appended to
If you want to save the results, or load the data from, elsewhere on your file system, please provide the full path name, using "forward slashes"
In the folder of where the .xlsx file is situated, all QC plots will be saved. Their name will end with the below 'tag'

In [6]:
rd_file = 'results_folder/RokDoc_SumsAverages_from_python.xlsx'

tag = 'my_test'

Modify the project_table.xlsx so that it points to the wells; Well_A, Well_B and Well_C

<img src="images/Annotation 2020-02-11 093608.png" />


Log names under each log types *P velocity*, *S velocity*, *Density*, *Porosity*, and *Volume* must be specified.

Multiple log names can be specified under each log type, but only one log per log type can be used in the statistics.

Therefore we need a table to specify which log to use under each log type


In [7]:
log_table = {'P velocity': 'vp_dry', 'S velocity': 'vs_dry', 'Density': 'rho_dry', 'Porosity': 'phie', 'Volume': 'vcl'}

Define the formations / intervals you like to calculate the statistics over.
 - 'name' is an arbitrary name used when writing the sums & averages to file
 - 'tops' is a list of formation / interval names of the top and base that defines the interval of interest. For the calculation to work, these names have to exist in the above defined tops_file, for the given wells. 

In [8]:
intervals_sands = [
            {'name': 'Sand H',
             'tops': ['Top H', 'Base H']},        
            {'name': 'Sand F',
             'tops': ['Top F', 'Base F']},
            {'name': 'Sand E',
             'tops': ['Top E', 'Top F']},
            {'name': 'Sand D',
             'tops': ['Top D', 'Top E']},
            {'name': 'Sand C',
             'tops': ['Top C', 'Top D']}
]
intervals_shales = [
    {'name': 'Shale C',
         'tops': ['Top C', 'Top D']},
    {'name': 'Shale G',
         'tops': ['Top G', 'Top H']}
]

Test to see if the above intervals are present in all wells

In [9]:
for intervals in [intervals_sands, intervals_shales]:
    for key in list(wells.keys()):
        for top_name in [x['tops'][0].upper() for x in intervals]:
            if top_name not in list(tops[key].keys()):
                print('Top: {}, not present in well {}'.format(top_name, key))
        for base_name in [x['tops'][1].upper() for x in intervals]:
            if base_name not in list(tops[key].keys()):
                print('Base: {}, not present in well {}'.format(base_name, key))

Define the cut offs that are used to classify the data (e.g. sands or shales).
The statistics will only be calculated within each interval, where the cut off is valid
The log names (e.g. *vcl*) corresponding to a log type (e.g. *Volume*) must exist in the .las files

In [10]:
cutoffs_sands = {'Volume': ['<', 0.5], 'Porosity': ['>', 0.1]}
cutoffs_shales = {'Volume': ['>', 0.5], 'Porosity': ['<', 0.1]}

Run the calculation of the statistics

In [12]:
calc_stats(wells, log_table, tops, intervals_sands, cutoffs_sands, 
              rokdoc_output=rd_file,
              working_dir=os.path.join(wp.working_dir, 'results_folder'),
              suffix=tag)

Interval: Sand H
 Well: WELL_A
   Interval Sand H, in well WELL_A, has the depth range: 2211.6 - 2364.9
 Well: WELL_B
   Interval Sand H, in well WELL_B, has the depth range: 2692.2 - 2770.1
 Well: WELL_C
   Interval Sand H, in well WELL_C, has the depth range: 2068.4 - 2092.0
Creating new RokDoc Sums and Averages file
Interval: Sand F
 Well: WELL_A
   Interval Sand F, in well WELL_A, has the depth range: 1984.0 - 2154.3
 Well: WELL_B
   Interval Sand F, in well WELL_B, has the depth range: 2530.3 - 2636.8
 Well: WELL_C
   Interval Sand F, is lacking in well WELL_C


C:\Users\mblixt\PycharmProjects\blixt_rp\utils\calc_stats.py:199: RuntimeWarning: Mean of empty slice
  mn = np.nanmean(results_per_well[this_well_name][key])
C:\Apps\Anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:1628: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)


Appending to existing RokDoc Sums and averages file
Interval: Sand E
 Well: WELL_A
   Interval Sand E, in well WELL_A, has the depth range: 1881.0 - 1983.9
 Well: WELL_B
   Interval Sand E, in well WELL_B, has the depth range: 2127.2 - 2501.3
 Well: WELL_C
   Interval Sand E, in well WELL_C, has the depth range: 1831.5 - 1970.4
Appending to existing RokDoc Sums and averages file
Interval: Sand D
 Well: WELL_A
   Interval Sand D, in well WELL_A, has the depth range: 1830.7 - 1860.8
 Well: WELL_B
   Interval Sand D, is lacking in well WELL_B
 Well: WELL_C
   Interval Sand D, is lacking in well WELL_C
Appending to existing RokDoc Sums and averages file
Interval: Sand C
 Well: WELL_A
   Interval Sand C, in well WELL_A, has the depth range: 1813.4 - 1825.6
 Well: WELL_B
   Interval Sand C, is lacking in well WELL_B
 Well: WELL_C
   Interval Sand C, is lacking in well WELL_C
Appending to existing RokDoc Sums and averages file


In [14]:
calc_stats(wells, log_table, tops, intervals_shales, cutoffs_shales, 
              rokdoc_output=rd_file,
              working_dir=os.path.join(wp.working_dir, 'results_folder'),
              suffix=tag)

Interval: Shale C
 Well: WELL_A
   Interval Shale C, in well WELL_A, has the depth range: 1585.1 - 1820.7
 Well: WELL_B
   Interval Shale C, in well WELL_B, has the depth range: 1787.8 - 1948.0
 Well: WELL_C
   Interval Shale C, in well WELL_C, has the depth range: 765.0 - 1763.9
Appending to existing RokDoc Sums and averages file
Interval: Shale G
 Well: WELL_A
   Interval Shale G, in well WELL_A, has the depth range: 2158.1 - 2211.0
 Well: WELL_B
   Interval Shale G, in well WELL_B, has the depth range: 2656.1 - 2691.9
 Well: WELL_C
   Interval Shale G, in well WELL_C, has the depth range: 2024.3 - 2067.4
Appending to existing RokDoc Sums and averages file
